In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
data_Fake = pd.read_csv('Fake.csv')
data_True = pd.read_csv('True.csv')

In [3]:
data_Fake.info()
data_True.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [4]:
data_Fake = data_Fake.drop(['title','subject','date'],axis = 1)

In [5]:
data_True = data_True.drop(['title','subject','date'],axis = 1)

In [6]:
data_Fake['class'] = 0
data_True['class'] = 1

In [7]:
data_all = pd.concat([data_Fake,data_True],axis = 0)

In [8]:
data_all = data_all.sample(frac=1)

In [9]:
data_all.reset_index(inplace = True)

In [10]:
data_all.drop('index',axis = 1,inplace = True)

In [11]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)  
    text = re.sub("\\W", ' ', text)  
    text = re.sub('https?://\S+|www\.\S+', '', text)  
    text = re.sub('<.*?>+', '', text) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  
    text = re.sub('\n', '', text)  
    text = re.sub('\w*\d\w*', '', text)
    return text


data_all['text'] = data_all['text'].apply(wordopt)

x = data_all['text']
y = data_all['class']


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train =  vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [13]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [14]:
pred_1r = LR.predict(xv_test)
LR.score (xv_test, y_test)

0.9865478841870824

In [15]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [16]:
DT.score (xv_test, y_test)

0.9953674832962138

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)
pred_gb = GB.predict(xv_test)
GB.score (xv_test, y_test)
print(classification_report (y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5804
           1       0.99      1.00      1.00      5421

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



In [18]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [19]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization. transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format (output_lable(pred_LR[0]),output_lable(pred_DT[0]),output_lable(pred_GB[0]),output_lable(pred_RF[0])))

In [20]:
news = str(input())
manual_testing(news)


 this is good news




LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News
